In [117]:
import re
import pandas as pd
import torch
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search

In [118]:
model = SentenceTransformer("sentence-transformers/gtr-t5-large")

In [119]:
df = pd.read_csv('../data/raw/foia_sample.csv')
df.rename(columns = {'DEPENDENCIA': 'dependencia', 'DESCRIPCIONSOLICITUD': 'solicitud'}, inplace = True)

In [120]:
df['dependencia'] = df['dependencia'].str.title()
df['solicitud'] = df['solicitud'].str.lower()
df['solicitud'] = df['solicitud'].apply(lambda x: re.sub(r'\n', '', x))
df

,dependencia,solicitud
0,Municipio De Asientos,1. deseo conocer si actualmente la administrac...
1,Patronato De La Feria Nacional De San Marcos,descripción solicitud
2,H.Congreso Del Estado De Aguascalientes,solicito se me proporcione copia de la versión...
3,Municipio De Pabellón De Arteaga,cuáles programas sociales de llevaron a cabo d...
4,Secretaría Desarrollo Social,solicito al instituto aguascalentense de las m...
...,...,...
2878,"Comision De Agua Potable, Alcantarillado Y San...",solicito copia de los contratos vigentes forma...
2879,Colegio De Estudios Científicos Y Tecnológicos...,solicito copia de los contratos vigentes forma...
2880,Colegio De Educación Profesional Técnica Del E...,solicito copia de los contratos vigentes forma...
2881,Centro De Conciliación Laboral Del Estado De A...,solicito copia de los contratos vigentes forma...


In [136]:
df['combined'] = "Solicitud: " + df['solicitud'] + "Dependencia:" + df['dependencia']
df.to_csv('../data/processed/foia_sample.csv', index=False)
df

,dependencia,solicitud,combined
0,Municipio De Asientos,1. deseo conocer si actualmente la administrac...,Solicitud: 1. deseo conocer si actualmente la ...
1,Patronato De La Feria Nacional De San Marcos,descripción solicitud,Solicitud: descripción solicitudDependencia:Pa...
2,H.Congreso Del Estado De Aguascalientes,solicito se me proporcione copia de la versión...,Solicitud: solicito se me proporcione copia de...
3,Municipio De Pabellón De Arteaga,cuáles programas sociales de llevaron a cabo d...,Solicitud: cuáles programas sociales de llevar...
4,Secretaría Desarrollo Social,solicito al instituto aguascalentense de las m...,Solicitud: solicito al instituto aguascalenten...
...,...,...,...
2878,"Comision De Agua Potable, Alcantarillado Y San...",solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...
2879,Colegio De Estudios Científicos Y Tecnológicos...,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...
2880,Colegio De Educación Profesional Técnica Del E...,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...
2881,Centro De Conciliación Laboral Del Estado De A...,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...


In [122]:
df_percent = df.sample(frac=0.8)
df_percent

,dependencia,solicitud,combined
784,Instituto Estatal Electoral Aguascalientes,cantidad que por concepto de sueldo recibe su ...,Solicitud: cantidad que por concepto de sueldo...
183,"Secretaría De Sustentabilidad, Medio Ambiente ...",descripción solicitud,Solicitud: descripción solicitudDependencia:Se...
1841,Instituto De Infraestructura Física Educativa ...,solicito información sobre su dependencia cuán...,Solicitud: solicito información sobre su depen...
1586,Instituto De Servicios De Salud Del Estado De ...,descripción solicitud,Solicitud: descripción solicitudDependencia:In...
795,Secretaría General De Gobierno,"solicito me informe, żcuántas víctimas directa...","Solicitud: solicito me informe, żcuántas vícti..."
...,...,...,...
619,Secretaría De Salud,solicito una relación en formato abierto (.xls...,Solicitud: solicito una relación en formato ab...
1115,Partido Verde Ecologista De México,descripción solicitud,Solicitud: descripción solicitudDependencia:Pa...
654,Instituto Municipal Aguascalentense Para La Cu...,"buenos días, quisiera saber a cuanto ascienden...","Solicitud: buenos días, quisiera saber a cuant..."
1193,Partido Del Trabajo,"buenos días, quisiera conocer su aviso de priv...","Solicitud: buenos días, quisiera conocer su av..."


In [145]:
df_percent.to_csv('../data/processed/train.csv', index=False)

In [123]:
texts = df_percent['combined'].to_list()
embeddings = model.encode(texts)

In [124]:
df_emb =pd.DataFrame(embeddings)
df_emb.to_csv('../data/processed/embeddings.csv', index=False)

In [125]:
dataset = Dataset.from_pandas(df_emb)

dataset_embeddings = torch.from_numpy(dataset.to_pandas().to_numpy()).to(torch.float)

In [146]:
df_rest = df.loc[~df.index.isin(df_percent.index)]
df_rest.to_csv('../data/processed/test.csv', index=False)
df_rest

,dependencia,solicitud,combined
2,H.Congreso Del Estado De Aguascalientes,solicito se me proporcione copia de la versión...,Solicitud: solicito se me proporcione copia de...
4,Secretaría Desarrollo Social,solicito al instituto aguascalentense de las m...,Solicitud: solicito al instituto aguascalenten...
6,Supremo Tribunal De Justicia Del Estado De Agu...,descripción solicitud,Solicitud: descripción solicitudDependencia:Su...
8,Colegio De Educación Profesional Técnica Del E...,descripción solicitud,Solicitud: descripción solicitudDependencia:Co...
9,Instituto De Servicios De Salud Del Estado De ...,necesito el expediente clínico de eliezer adbe...,Solicitud: necesito el expediente clínico de e...
...,...,...,...
2857,Instituto Aguascalentense De Las Mujeres,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...
2858,Instituto Aguascalentense De La Juventud,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...
2859,H. Congreso Del Estado De Aguascalientes,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...
2861,Despacho Del Ejecutivo Estatal,solicito copia de los contratos vigentes forma...,Solicitud: solicito copia de los contratos vig...


### Evaluate

In [127]:
example_eval = df_rest['combined'].sample(n=1)
index = example_eval.index
idx = index[0]
idx

1832

In [128]:
df.iloc[idx]

dependencia            Universidad Politécnica De Aguascalientes
solicitud      por medio de esta solicitud de acceso a la inf...
combined       Solicitud: por medio de esta solicitud de acce...
Name: 1832, dtype: object

In [129]:
request = example_eval.to_string(index=False)
request

'Solicitud: por medio de esta solicitud de acces...'

In [130]:

output = model.encode(request)
query_embeddings = torch.FloatTensor(output)

In [131]:
hits = semantic_search(query_embeddings, dataset_embeddings, top_k=3)
hits

[[{'corpus_id': 349, 'score': 0.8444914817810059},
  {'corpus_id': 2028, 'score': 0.8444793820381165},
  {'corpus_id': 963, 'score': 0.8279389142990112}]]

In [132]:
id1 = hits[0][0]['corpus_id']
id2 = hits[0][1]['corpus_id']
id3 = hits[0][2]['corpus_id']

In [133]:
df.iloc[id1]

dependencia             Instituto De Educación De Aguascalientes
solicitud      solicito del c. víctor hugo cervantes nájera l...
combined       Solicitud: solicito del c. víctor hugo cervant...
Name: 349, dtype: object

In [134]:
df.iloc[id2]

dependencia                    Coordinación General De Movilidad
solicitud      quiero solicitarle la siguiente información:- ...
combined       Solicitud: quiero solicitarle la siguiente inf...
Name: 2028, dtype: object

In [135]:
df.iloc[id3]

dependencia            Universidad Tecnológica De Aguascalientes
solicitud      buenos días, quisiera saber su protocolo de se...
combined       Solicitud: buenos días, quisiera saber su prot...
Name: 963, dtype: object

In [144]:
df.iloc[id3].str.split(pat="/")[0]

['Universidad Tecnológica De Aguascalientes']